In [1]:
import torch
from torch import nn
from torch.nn.parameter import Parameter

In [2]:
class eca_layer(nn.Module):
    """Constructs a ECA module.

    Args:
        channel: Number of channels of the input feature map
        k_size: Adaptive selection of kernel size
    """
    def __init__(self, channel, k_size = 3) -> None:
        super().__init__()
        self.avg_pool = nn.AdaptiveMaxPool2d(1)
        self.conv = nn.Conv1d(1, 1, kernel_size=k_size, padding=(k_size - 1)//2, bias=False)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        # 全局空间信息的特征描述符
        # feature descriptor on the global spatial information
        y = self.avg_pool(x)
        # two different branches of ECA module
        y = self.conv(y.squeeze(-1).transpose(-1, -2)).transpose(-1, -2).unsqueeze(-1)
        
        # multi scale information fusion
        y = self.sigmoid(y)
        
        return x * y.expand_as(x)

In [14]:
class eca_block(nn.Module):
    """Constructs a ECA module.

    Args:
        channel: Number of channels of the input feature map
        k_size: Adaptive selection of kernel size
    """
    def __init__(self, channel, k_size = 3) -> None:
        super().__init__()
        self.avg_pool = nn.AdaptiveMaxPool2d(1)
        self.conv = nn.Conv1d(1, 1, kernel_size=k_size, padding=(k_size - 1)//2, bias=False)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        # 全局空间信息的特征描述符
        # feature descriptor on the global spatial information
        print(f'x:{x.size()}')
        y = self.avg_pool(x)
        print(f'y:{y.size()} ')
        # two different branches of ECA module
        print(f'y.squeeze:{y.squeeze(-1).size()} ')
        print(f'y.transpose:{y.squeeze(-1).transpose(-1, -2).size()} ')
        y = self.conv(y.squeeze(-1).transpose(-1, -2)).transpose(-1, -2).unsqueeze(-1)
        
        print(f'y_conv:{y.size()}')
        
        # multi scale information fusion
        y = self.sigmoid(y)
        
        return x * y.expand_as(x)

In [15]:
x = torch.randn(1, 2, 3, 4)
mod = eca_block(2)
y = mod(x)

x:torch.Size([1, 2, 3, 4])
y:torch.Size([1, 2, 1, 1]) 
y.squeeze:torch.Size([1, 2, 1]) 
y.transpose:torch.Size([1, 1, 2]) 
y_conv:torch.Size([1, 2, 1, 1])


In [5]:
def mean_channels(F):
    assert(F.dim() == 4)
    spatial_sum = F.sum(3, keepdim=True).sum(2, keepdim=True)
    return spatial_sum / (F.size(2) * F.size(3))

def stdv_channels(F):
    assert (F.dim() == 4)
    F_mean = mean_channels(F)
    F_variance = (F - F_mean).pow(2).sum(3, keepdim=True).sum(2, keepdim=True) / (F.size(2) * F.size(3))
    return F_variance.pow(0.5)

In [6]:
class test(nn.Module):
    def __init__(self, channel, reduction=16) -> None:
        super().__init__()
        
        self.contrast = stdv_channels
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        
        self.s1  = nn.Sequential(
            nn.Conv2d(channel, channel , (2, 1), padding=0, bias=False),
            nn.ReLU(inplace=True),
        )
        
        self.s2 =  nn.Sequential(
            nn.Conv2d(channel , channel, 1, padding=0, bias=True),
            nn.Sigmoid()
        )

    def forward(self, x):
        y = torch.cat([self.contrast(x), self.avg_pool(x)], dim = 2)
        print(f'y:{y.size()}\n{y}')
        y = self.s1(y)
        print(f'y1:{y.size()}\n{y}')
        y = self.s2(y)
        print(f'y2:{y.size()}\n{y}')


In [7]:
mode = test(4)
x = torch.rand(1, 4, 16, 16)
#print(f'x:{x}')
mode(x)

y:torch.Size([1, 4, 2, 1])
tensor([[[[0.2838],
          [0.5030]],

         [[0.2913],
          [0.4834]],

         [[0.2959],
          [0.4989]],

         [[0.2783],
          [0.5059]]]])
y1:torch.Size([1, 4, 1, 1])
tensor([[[[0.]],

         [[0.]],

         [[0.]],

         [[0.]]]], grad_fn=<ReluBackward0>)
y2:torch.Size([1, 4, 1, 1])
tensor([[[[0.4952]],

         [[0.5008]],

         [[0.5179]],

         [[0.5080]]]], grad_fn=<SigmoidBackward0>)


In [8]:
class CCALayer(nn.Module):
    def __init__(self, channel, reduction=16):
        super(CCALayer, self).__init__()

        self.contrast = stdv_channels
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.s1  = nn.Sequential(
            nn.Conv2d(channel, channel , 1, padding=0, bias=False),
            nn.ReLU(inplace=True),
        )
        
        self.s2 =  nn.Sequential(
            nn.Conv2d(channel , channel, 1, padding=0, bias=True),
            nn.Sigmoid()
        )

    def forward(self, x):
        y = self.contrast(x) + self.avg_pool(x)
        print(f'y:{y.size()}\n{y}')
        y = self.s1(y)
        print(f'y1:{y.size()}\n{y}')
        y = self.s2(y)
        print(f'y2:{y.size()}\n{y}')
        return x * y

In [9]:
mode = CCALayer(4)
x = torch.rand(1, 4, 16, 16)
# print(f'x:{x}')
y = mode(x)
# print(1)

y:torch.Size([1, 4, 1, 1])
tensor([[[[0.8061]],

         [[0.7731]],

         [[0.8092]],

         [[0.7989]]]])
y1:torch.Size([1, 4, 1, 1])
tensor([[[[0.1048]],

         [[0.2161]],

         [[0.1172]],

         [[0.0951]]]], grad_fn=<ReluBackward0>)
y2:torch.Size([1, 4, 1, 1])
tensor([[[[0.4933]],

         [[0.5317]],

         [[0.5302]],

         [[0.4120]]]], grad_fn=<SigmoidBackward0>)
